In [5]:
from to_magnitudes import get_physical_by_id, get_m_ref, isis_flux_to_mag
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def flux_to_magnitudes(set_name, star_id, reference_fit, star_lc, physical_coords_file): 
    # leer archivo con pandas (más robusto que np.loadtxt)
    df = pd.read_csv(star_lc, delim_whitespace=True, header=None)

    # delta_flux está en la columna 1 (segunda columna)
    delta_flux = df[1].to_numpy(dtype=float)
    
    # coordenadas físicas
    x_physical, y_physical = get_physical_by_id(physical_coords_file, star_id)
    
    # magnitud de referencia
    m_ref, flux_ref = get_m_ref(reference_fit, x_physical, y_physical)
    #print(f"m_ref = {m_ref:.3f} (flux = {flux_ref:.1f})")
    
    # conversión a magnitudes instrumentales
    mag_instr = isis_flux_to_mag(delta_flux, m_ref, C0=25)
    
    # agregar magnitud como nueva columna
    df[6] = mag_instr
    
    # guardar en mismo set
    output_path = f'{set_name}/lcs/lc{set_name[8:]}_{star_id}_mag.data'
    df.to_csv(output_path, sep="\t", index=False, header=False)
    #print(f"✅ Guardado: {output_path}")
    return df

hacemos una lista de nuestros sets

In [6]:
sets = !ls NGC6426_* -d
sets

['NGC6426_2407_I',
 'NGC6426_2407_V',
 'NGC6426_2505_I',
 'NGC6426_2505_V',
 'NGC6426_2506A_I',
 'NGC6426_2506A_V',
 'NGC6426_2506B_I',
 'NGC6426_2506B_V',
 'NGC6426_2508_I',
 'NGC6426_2508_V']

obtenemos una lista de los ids del catalogo de coordenadas `coordinates_NGC6426.dat` que nos pasó Raul y lo guardamos en `todas`

In [ ]:
todas = !awk '{print $1}' NGC6426/coordinates_NGC6426.dat
print(len(todas))

todas[:5]

1373


['6', '7', '8', '9', '10']

In [8]:
import logging
from collections import Counter
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="astropy")

# configurar log (solo a archivo, no en pantalla)
logging.basicConfig(
    filename="NGC6426/errores.log",
    filemode="a",
    level=logging.ERROR,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

total = len(sets) * len(todas)
procesados = 0
errores = Counter()

for set in sets: 
    for star_id in todas: 
        set_id = set[8:]
        reference_fit = f'{set}/ref.fits'
        star_lc = f'{set}/lcs/lc{star_id}.data'
        physical_coords_file = f'{set}/phot_{set_id}.data'

        try:
            flux_to_magnitudes(
                set_name=set,
                star_id=star_id,
                reference_fit=reference_fit,
                star_lc=star_lc,
                physical_coords_file=physical_coords_file
            )
        except FileNotFoundError:
            errores["FileNotFound"] += 1
            logging.error(f"{star_lc} no existe")
        except Exception as e:
            errores["Otros"] += 1
            logging.error(f"Error procesando {star_lc}: {e}")
        finally:
            procesados += 1
            if procesados % 20 == 0:  # actualiza cada 20 procesos
                pct = (procesados / total) * 100
                print(f"[{pct:.1f}%] Procesados: {procesados}/{total} | "
                      f"Errores: {sum(errores.values())} "
                      f"(FileNotFound={errores['FileNotFound']}, Otros={errores['Otros']})")

NGC6426_2407_I/lcs/lc6.data no existe
NGC6426_2407_I/lcs/lc7.data no existe
NGC6426_2407_I/lcs/lc8.data no existe
NGC6426_2407_I/lcs/lc9.data no existe
NGC6426_2407_I/lcs/lc10.data no existe
NGC6426_2407_I/lcs/lc11.data no existe
NGC6426_2407_I/lcs/lc12.data no existe
NGC6426_2407_I/lcs/lc14.data no existe


[0.1%] Procesados: 20/13730 | Errores: 8 (FileNotFound=8, Otros=0)
NGC6426_2407_I/lcs/lc30.data no existe
[0.3%] Procesados: 40/13730 | Errores: 9 (FileNotFound=9, Otros=0)
NGC6426_2407_I/lcs/lc63.data no existe
[0.4%] Procesados: 60/13730 | Errores: 10 (FileNotFound=10, Otros=0)
[0.6%] Procesados: 80/13730 | Errores: 10 (FileNotFound=10, Otros=0)
NGC6426_2407_I/lcs/lc90.data no existe
[0.7%] Procesados: 100/13730 | Errores: 11 (FileNotFound=11, Otros=0)
NGC6426_2407_I/lcs/lc115.data no existe
[0.9%] Procesados: 120/13730 | Errores: 12 (FileNotFound=12, Otros=0)
NGC6426_2407_I/lcs/lc144.data no existe
[1.0%] Procesados: 140/13730 | Errores: 13 (FileNotFound=13, Otros=0)
[1.2%] Procesados: 160/13730 | Errores: 13 (FileNotFound=13, Otros=0)
[1.3%] Procesados: 180/13730 | Errores: 13 (FileNotFound=13, Otros=0)
NGC6426_2407_I/lcs/lc200.data no existe
[1.5%] Procesados: 200/13730 | Errores: 14 (FileNotFound=14, Otros=0)
[1.6%] Procesados: 220/13730 | Errores: 14 (FileNotFound=14, Otros=0)
N

### Ejecutamos el script `./carpeta_por_id.sh`. 

Esto crea los directorios 

`NGC6426/lcs/V/` 

y 

`NGC6426/lcs/I/`, 

ademas de una carpeta para cada `id` donde copia todas las curvas de luz de la misma estrella y filtro a través de los distintos sets.

In [17]:
I_list = !ls NGC6426/lcs/I 
V_list = !ls NGC6426/lcs/V

len(V_list)

1373

### Concatenamos 
todas las curvas de luz de la misma estrella de acuerdo a su directorio dentro de `NGC6426/lcs/<filtro>/<id>`

Los resultados se guardan en los directorios 

`NGC6426/lcs/V_full` 

y 

`NGC6426/lcs/I_full`

In [20]:
import pandas as pd
!mkdir NGC6426/lcs/V_full/
for lc in V_list: 
    to_merge = !ls NGC6426/lcs/V/{lc}/*
    #combina todos los archivos de la lista
    combinado_csv = pd.concat([pd.read_csv(f, delim_whitespace=True, header=None) for f in to_merge])
    #exporta a csv
    combinado_csv.to_csv(f"NGC6426/lcs/V_full/lc{lc}_V.data", sep="\t", index=False, header=False)

lo mismo solo que añadimos un contador porque tarda un poco el anterior: 

In [23]:
!mkdir NGC6426/lcs/I_full/
i = 0 
for lc in I_list: 
    i = i+1
    to_merge = !ls NGC6426/lcs/I/{lc}/*
    #combina todos los archivos de la lista
    combinado_csv = pd.concat([pd.read_csv(f, delim_whitespace=True, header=None) for f in to_merge])
    #exporta a csv
    combinado_csv.to_csv(f"NGC6426/lcs/I_full/lc{lc}_I.data", sep="\t", index=False, header=False)
    print(i, '/', len(I_list))

mkdir: cannot create directory ‘NGC6426/lcs/I_full/’: File exists
1 / 1373
2 / 1373
3 / 1373
4 / 1373
5 / 1373
6 / 1373
7 / 1373
8 / 1373
9 / 1373
10 / 1373
11 / 1373
12 / 1373
13 / 1373
14 / 1373
15 / 1373
16 / 1373
17 / 1373
18 / 1373
19 / 1373
20 / 1373
21 / 1373
22 / 1373
23 / 1373
24 / 1373
25 / 1373
26 / 1373
27 / 1373
28 / 1373
29 / 1373
30 / 1373
31 / 1373
32 / 1373
33 / 1373
34 / 1373
35 / 1373
36 / 1373
37 / 1373
38 / 1373
39 / 1373
40 / 1373
41 / 1373
42 / 1373
43 / 1373
44 / 1373
45 / 1373
46 / 1373
47 / 1373
48 / 1373
49 / 1373
50 / 1373
51 / 1373
52 / 1373
53 / 1373
54 / 1373
55 / 1373
56 / 1373
57 / 1373
58 / 1373
59 / 1373
60 / 1373
61 / 1373
62 / 1373
63 / 1373
64 / 1373
65 / 1373
66 / 1373
67 / 1373
68 / 1373
69 / 1373
70 / 1373
71 / 1373
72 / 1373
73 / 1373
74 / 1373
75 / 1373
76 / 1373
77 / 1373
78 / 1373
79 / 1373
80 / 1373
81 / 1373
82 / 1373
83 / 1373
84 / 1373
85 / 1373
86 / 1373
87 / 1373
88 / 1373
89 / 1373
90 / 1373
91 / 1373
92 / 1373
93 / 1373
94 / 1373
95 